# Objetivo

Aplicar o K Nearest Neighbors Na base toda. 

# Método

A partir do conjunto reduzido de features (39) 100k rows, aplicar:
- fillna <b> OK </b>

- LabelEncoder (ou algo do tipo) <b> OK </b>
- Nearest Neighbors OK
    

# Leitura dos arquivos

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Leitura da lista de features a excluir

# Carregamento Clientes

## Redução no número de features
Só rodar caso seja necessário filtrar mais vezes

In [3]:
market = pd.read_csv('estaticos_market.csv')
port1 = pd.read_csv('estaticos_portfolio1.csv')

In [4]:
exclude_df = pd.read_csv('exclude_variables_generated_from_profiling.csv')
exclude_variables = exclude_df.values.squeeze().tolist()
len(exclude_variables)


143

In [5]:
port1.shape

(555, 182)

In [6]:
market.drop(columns=exclude_variables, inplace= True)
market.drop(columns=['Unnamed: 0'], inplace= True)
market.set_index('id',inplace=True)
market.shape

(462298, 37)

In [7]:
port1.drop(columns=exclude_variables, inplace= True)
port1.drop(columns=['Unnamed: 0'], inplace= True)
port1.set_index('id',inplace=True)
port1.shape

(555, 37)

In [ ]:
#p1.to_csv('estaticos_49_features.csv')

# Managing NaNs


Para usar nos dados do portfólio:
- passar as strings para valor numérico
- passar bool para int
- fazer fillna para todos
- - em categoricas -> other
- - em numericas -> 0

Estamos usando o LabelEncoder

In [8]:
fill_dict = market.dtypes.to_dict()
cat2bool = ['fl_optante_simei','fl_optante_simples','fl_passivel_iss']

for feature in fill_dict:
    if(fill_dict[feature] == bool):
        fill_dict[feature] = False
    elif(fill_dict[feature] == object):
        fill_dict[feature] = 'other'
    else:
        fill_dict[feature] = 0
    if feature in cat2bool:
        fill_dict[feature] = False
        
fill_dict

{'fl_matriz': False,
 'de_natureza_juridica': 'other',
 'sg_uf': 'other',
 'natureza_juridica_macro': 'other',
 'de_ramo': 'other',
 'setor': 'other',
 'idade_empresa_anos': 0,
 'idade_emp_cat': 'other',
 'fl_me': False,
 'fl_sa': False,
 'fl_mei': False,
 'fl_ltda': False,
 'dt_situacao': 'other',
 'fl_st_especial': False,
 'fl_email': False,
 'fl_telefone': False,
 'fl_rm': 'other',
 'nm_divisao': 'other',
 'nm_segmento': 'other',
 'fl_optante_simples': False,
 'fl_optante_simei': False,
 'sg_uf_matriz': 'other',
 'de_saude_tributaria': 'other',
 'de_saude_rescencia': 'other',
 'nu_meses_rescencia': 0,
 'de_nivel_atividade': 'other',
 'empsetorcensitariofaixarendapopulacao': 0,
 'nm_meso_regiao': 'other',
 'nm_micro_regiao': 'other',
 'fl_passivel_iss': False,
 'qt_socios': 0,
 'idade_media_socios': 0,
 'qt_socios_feminino': 0,
 'de_faixa_faturamento_estimado': 'other',
 'de_faixa_faturamento_estimado_grupo': 'other',
 'vl_faturamento_estimado_aux': 0,
 'vl_faturamento_estimado_grupo

In [9]:
market.fillna(value=fill_dict, inplace=True)
market[cat2bool] = market[cat2bool].astype(bool)

bool2numeric = market.columns[market.dtypes == bool]
market[bool2numeric] = market[bool2numeric].astype(int)

In [10]:
#market_p1.head(2)

## Encoding

In [11]:
cat = market.columns[market.dtypes == object]
not_cat = market.columns[market.dtypes != object]

In [12]:

# Usando LabelEncoder
#https://chrisalbon.com/machine_learning/preprocessing_structured_data/convert_pandas_categorical_column_into_integers_for_scikit-learn/
from sklearn import preprocessing
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()

In [13]:
market_le = pd.DataFrame()
market_le[cat] = market[cat].apply(lambda col: le.fit_transform(col))
market_encoded = pd.concat([market[not_cat],market_le], axis =1)
market_encoded.dtypes

fl_matriz                                  int64
idade_empresa_anos                       float64
fl_me                                      int64
fl_sa                                      int64
fl_mei                                     int64
fl_ltda                                    int64
fl_st_especial                             int64
fl_email                                   int64
fl_telefone                                int64
fl_optante_simples                         int64
fl_optante_simei                           int64
nu_meses_rescencia                       float64
empsetorcensitariofaixarendapopulacao    float64
fl_passivel_iss                            int64
qt_socios                                float64
idade_media_socios                       float64
qt_socios_feminino                       float64
vl_faturamento_estimado_aux              float64
vl_faturamento_estimado_grupo_aux        float64
de_natureza_juridica                       int64
sg_uf               

Aqui fiz fiz um novo dataset para os ids do portifolio1 e o merge com os dados já aplicados o LabelEncoder

In [14]:
port1_le = pd.DataFrame(index=port1.index)
port1_le = market_encoded.merge(port1_le, how='right',left_index=True,right_index=True)


In [16]:
port1_le.head(3)

,fl_matriz,idade_empresa_anos,fl_me,fl_sa,fl_mei,fl_ltda,fl_st_especial,fl_email,fl_telefone,fl_optante_simples,...,nm_divisao,nm_segmento,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,de_nivel_atividade,nm_meso_regiao,nm_micro_regiao,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo
id,,,,,,,,,,,,,,,,,,,,,
dabe79bec87c88ae04e869bf6bd321ee5e1893cecf66255be9ea861a9e5841a9,1,0.646575,0,0,1,0,0,0,0,0,...,30,9,16,2,4,1,14,61,10,10
32e5f4e10932153a7ba869cb0386e7e02d49d2461046b866411556d2af150702,0,6.586301,0,0,0,0,0,1,1,0,...,30,9,9,4,0,2,3,44,10,7
a95d6f30bba445bd3d6b0c5b36f865b38ec01d1733609095d359649cfedf15d1,1,8.010959,0,0,1,0,0,0,0,1,...,30,9,9,1,0,2,8,0,1,1


# Nearest Neighbors

Como não temos o que prever, usamos o treinamento não supervisionado do Nearest Neighbors



Apenas amostra por enquanto

Precisa escolher o numero de componentes a indicar

In [17]:
market_sample = market_encoded.sample(n=10000)

In [18]:
from sklearn.neighbors import NearestNeighbors

nearest = NearestNeighbors()

nearest.fit(market_sample)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

Dict para a lista de vizinhos mais próximos

In [26]:
neighbors_list = {}

for row in range(port1_le.shape[0]):
    #print(row)
    neighbors_list[row] = nearest.kneighbors(port1_le.iloc[[row]].values)

In [24]:
nearest.kneighbors(port1_le.iloc[[2]].values)

(array([[33.75838096, 39.54600935, 50.29927343, 55.69418003, 72.04832592]]),
 array([[9582, 2156, 3699, 1931, 2527]]))

A resposta é uma lista da distância e uma lsita de qual a componente. Precisa confirmar se a componente é a linha do Dataset

In [27]:
neighbors_list

{0: (array([[ 60.12274005,  69.53971155,  83.72121579,  93.19553409,
          101.55240751]]), array([[9639,  460, 9523, 2821, 8979]])),
 1: (array([[45213.79031159, 49096.81949266, 49104.65875102, 49109.83483867,
          49125.23103578]]), array([[7435, 9792, 2723, 7568, 5992]])),
 2: (array([[33.75838096, 39.54600935, 50.29927343, 55.69418003, 72.04832592]]),
  array([[9582, 2156, 3699, 1931, 2527]])),
 3: (array([[ 29.4149117 , 141.88807226, 170.81264943, 179.26665077,
          197.34518539]]), array([[4047, 1424, 4299, 6300,  228]])),
 4: (array([[ 577348.77621594,  621277.17148271,  704857.60694474,
          1019551.094203  , 1036295.65715562]]),
  array([[9144,  597, 8716, 1254, 6003]])),
 5: (array([[ 92.64851538,  92.86519318,  94.29061779, 106.76829196,
          117.26986957]]), array([[5268, 6295, 2086, 9034, 4837]])),
 6: (array([[25.81089347, 30.06659276, 35.48241073, 41.13406638, 41.54531551]]),
  array([[3795, 1831, 8495, 7508,  191]])),
 7: (array([[37.77895175, 47

## Próximas etapas

Agora com o NN funcionando, precisa confrontar as sugestões para saber qual é o mais sugerido